*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-03-03 07:39:43--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  82.4MB/s    in 0.3s    

2022-03-03 07:39:43 (82.4 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [9]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [26]:
df_ratings = df_ratings[df_ratings['user'].isin((df_ratings['user'].value_counts() > 200).index) & df_ratings['isbn'].isin((df_ratings['isbn'].value_counts() > 100).index)]

In [5]:
df = pd.DataFrame({'angles': [0, 3, 4],
                   'degrees': [360, 180, 360]},
                  index=['circle', 'triangle', 'rectangle'])
print(df)
(df.sub(df.iloc[0], axis=1) ** 2).sum(axis=1).sort_values()

           angles  degrees
circle          0      360
triangle        3      180
rectangle       4      360


circle           0
rectangle       16
triangle     32409
dtype: int64

In [16]:
df_ratings['rating'].value_counts()

0.0     716109
8.0     103736
10.0     78610
7.0      76457
9.0      67541
5.0      50974
6.0      36924
4.0       8904
3.0       5996
2.0       2759
1.0       1770
Name: rating, dtype: int64

In [27]:
df = pd.merge(df_books, df_ratings, on='isbn')
df.head()

# get cooccurence matrix :: book title -> users who rated that book -> books rated by users -> matrix
def get_cooccurence_matrix(title):
  isbn = df_books[df_books['title'] == title]['isbn']
  # print('isbn:', isbn.iloc[0])
  users = df_ratings[df_ratings['isbn'] == isbn.iloc[0]]['user'].unique()
  # print('users:', users[:5])
  books = df_ratings[df_ratings['user'].isin(users)]
  # print(books)
  m = books.pivot_table(index='isbn', columns='user', values='rating').fillna(0)
  # print(m)
  print("Non zero values in cooccurence_matrix :%d" % np.count_nonzero(m))
  return m




def get_recommends(book = ""):
  cooccurence_matrix = get_cooccurence_matrix(book)

  isbn = df_books[df_books['title'] == book]['isbn']

  x = cooccurence_matrix.sub(cooccurence_matrix.loc[isbn.iloc[0]], axis=1).pow(2).sum(axis=1)

  print('refs:')
  for t in [
            book,
            'Catch 22',
            'The Witching Hour (Lives of the Mayfair Witches)',
            'Interview with the Vampire',
            'The Tale of the Body Thief (Vampire Chronicles (Paperback))',
            'The Vampire Lestat (Vampire Chronicles, Book II)'
  ]:
    isbn_x = df_books[df_books['title'] == t]['isbn']
    print(t, x[isbn_x.iloc[0]])

  # isbn_x = df_books[df_books['title'] == 'The Witching Hour (Lives of the Mayfair Witches)']['isbn']
  # print('The Witching Hour (Lives of the Mayfair Witches)', x[isbn_x.iloc[0]])

  # isbn_x = df_books[df_books['title'] == 'Interview with the Vampire']['isbn']
  # print('Interview with the Vampire', x[isbn_x.iloc[0]])

  x_book = cooccurence_matrix.loc[isbn.iloc[0]]
  top_n = (cooccurence_matrix.sub(x_book, axis=1) ** 2).sum(axis=1).sort_values()
  print()
  print('norms:')
  for i in range(1, 6):
    print(df_books[df_books['isbn'] == top_n.index[i]]['title'].iloc[0], top_n.iloc[i])
  
  model_knn = NearestNeighbors(algorithm='brute', metric='cosine')
  model_knn.fit(cooccurence_matrix)
  distances, indices = model_knn.kneighbors(cooccurence_matrix.loc[isbn.iloc[0]].values.reshape(1, -1), n_neighbors=6)
  
  print()
  print('kneighbors:')
  for i, d in zip(indices[0][1:], distances[0][1:]):
    print(df_books[df_books['isbn'] == cooccurence_matrix.index[i]]['title'].iloc[0], d)

  return [book, [[df_books[df_books['isbn'] == cooccurence_matrix.index[i]]['title'].iloc[0], d] for i, d in zip(indices[0], distances[0])][1:]]

# [
#   'The Queen of the Damned (Vampire Chronicles (Paperback))',
#   [
#     ['Catch 22', 0.793983519077301], 
#     ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
#     ['Interview with the Vampire', 0.7345068454742432],
#     ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
#     ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
#   ]
# ]
books = get_recommends('The Queen of the Damned (Vampire Chronicles (Paperback))')

Non zero values in cooccurence_matrix :25069
refs:
The Queen of the Damned (Vampire Chronicles (Paperback)) 0.0
Catch 22 5639.0
The Witching Hour (Lives of the Mayfair Witches) 5405.0
Interview with the Vampire 4627.0
The Tale of the Body Thief (Vampire Chronicles (Paperback)) 3978.0
The Vampire Lestat (Vampire Chronicles, Book II) 4148.0

norms:
The Tale of the Body Thief (Vampire Chronicles (Paperback)) 3978.0
The Vampire Lestat (Vampire Chronicles, Book II) 4148.0
Interview with the Vampire 4627.0
Memnoch the Devil (Vampire Chronicles, No 5) 4944.0
Pandora: New Tales of the Vampires (New Tales of the Vampires) 5290.0

kneighbors:
The Vampire Lestat (Vampire Chronicles, Book II) 0.40147185
The Tale of the Body Thief (Vampire Chronicles (Paperback)) 0.42259222
Interview with the Vampire 0.46161252
Memnoch the Devil (Vampire Chronicles, No 5) 0.60804343
Pandora: New Tales of the Vampires (New Tales of the Vampires) 0.69310725


Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [7]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

Non zero values in cooccurence_matrix :40271
refs:
Where the Heart Is (Oprah's Book Club (Paperback)) 0.0
Catch 22 20080.0
The Witching Hour (Lives of the Mayfair Witches) 20282.0
Interview with the Vampire 19636.0
The Tale of the Body Thief (Vampire Chronicles (Paperback)) 20174.0
The Vampire Lestat (Vampire Chronicles, Book II) 20034.0

norms:
The Lovely Bones: A Novel 19105.0
Bridget Jones's Diary 19369.0
She's Come Undone (Oprah's Book Club) 19371.0
Interview with the Vampire 19636.0
Life of Pi 19661.0

kneighbors:
The Lovely Bones: A Novel 0.7053724
Bridget Jones's Diary 0.78692526
She's Come Undone (Oprah's Book Club) 0.7917692
The Pilot's Wife : A Novel 0.79883105
Divine Secrets of the Ya-Ya Sisterhood: A Novel 0.80180484
["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', 0.7053724], ["Bridget Jones's Diary", 0.78692526], ["She's Come Undone (Oprah's Book Club)", 0.7917692], ["The Pilot's Wife : A Novel", 0.79883105], ['Divine Secrets of the Ya